In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import unidecode
import os

In [19]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#### Funciones

In [20]:
def doClean(a):   
    a = str(a)
    # Quitar las expresiones tras la barra /
    a = re.sub('\/.*', '', a)
    # Quitar las aclaraciones entre paréntesis
    a = re.sub('\(.*\)', '', a)
    # Quitar acentos 
    a = unidecode.unidecode(a)
    # Quitar aquello que no sean palabras o cosa que se le parezca
    a = re.sub('\W', ' ', a) 
    # Quitar espacios extra en caso de haber
    a = re.sub('\s+', ' ', a)
    # Pasar el texto a minúsulas
    a = a.lower() 
    a = a.lstrip()
    # Quitar espacio derecha
    a = a.rstrip()
    a = re.sub(' ', '_', a)
    #a = a.strip() 
    
    return a

In [21]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=1):
    """
    df_1 is the left table to join
    df_2 is the right table to join
    key1 is the key column of the left table
    key2 is the key column of the right table
    threshold is how close the matches should be to return a match, based on Levenshtein distance
    limit is the amount of matches that will get returned, these are sorted high to low
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [94]:
def union_agrup(df_maestro, df_union, agrup_maestro, agrup_union, threshold=85, threshold_mun=90):
    
    count_maestro = df_maestro.shape[0]
    count_union = df_union.shape[0]
    print('num registros maestro: {}'.format(count_maestro))
    print('num registros fichero para unir: {}'.format(count_union))
    
    #fuzzy por prov/ccaa
    maestro_agrup = pd.DataFrame(df_maestro[agrup_maestro].drop_duplicates())
    print('num agrupaciones maestro: {}'.format(maestro_agrup.shape))
    union_agrup = pd.DataFrame(df_union[agrup_union].drop_duplicates())
    print('num agrupaciones union: {}'.format(union_agrup.shape))

    agrup_match = fuzzy_merge(maestro_agrup, union_agrup, agrup_maestro, agrup_union, threshold=threshold)
    print('num agrupaciones tras match: {}'.format(agrup_match.shape))
    num_matches = (agrup_match[agrup_match['matches']!='']['matches'].nunique())
    if (num_matches != agrup_match[agrup_maestro].nunique()):
        print('no se han cruzado todos los valores:')
        print(agrup_match[agrup_match['matches']==''])
        
        #se genera un nuevo df con el match y el fichero de union
    df_union_agrup_match = pd.merge(df_union, agrup_match, left_on=agrup_union, right_on='matches', how='right')
    if (count_union != df_union_agrup_match.shape[0]):
        print('el fichero de union no dispone de todos los valores para cruzar')    

    print('columnas final: {}'.format(df_union_agrup_match.columns))
        #borramos columnas que sobran
    df_union_agrup_match.drop(columns=['matches'], inplace=True)
    if agrup_maestro != agrup_union:
        df_union_agrup_match.drop(columns=agrup_union, inplace=True)
    print('num registros tras union: {}'.format(df_union_agrup_match.shape))
    
    return df_union_agrup_match

In [124]:
def union_solo_muni(df_maestro, df_union, muni_maestro, muni_union, threshold=85):
    
    count_maestro = df_maestro.shape[0]
    count_union = df_union.shape[0]
    print('num registros maestro: {}'.format(count_maestro))
    print('num registros fichero para unir: {}'.format(count_union))
    
    #fuzzy por municipio
    muni_match = fuzzy_merge(df_maestro, df_union, muni_maestro, muni_union, threshold=threshold)
    print('num agrupaciones tras match: {}'.format(muni_match.shape))
    num_matches = (muni_match[muni_match['matches']!='']['matches'].nunique())
    if (num_matches != muni_match[muni_maestro].nunique()):
        print('no se han cruzado todos los valores:')
        
        #se genera un nuevo df con el match y el fichero de union
    df_union_muni_match = pd.merge(df_union, muni_match, left_on=muni_union, right_on='matches', how='right')
    if (count_union != df_union_muni_match.shape[0]):
        print('el fichero de union no dispone de todos los valores para cruzar')    

    print('columnas final: {}'.format(df_union_muni_match.columns))
        #borramos columnas que sobran
    df_union_muni_match.drop(columns=['matches'], inplace=True)
    if muni_maestro != muni_union:
        df_union_muni_match.drop(columns=muni_union, inplace=True)
    print('num registros tras union: {}'.format(df_union_muni_match.shape))
    
    return df_union_muni_match

In [23]:
def union_muni(df_maestro, df_union, muni_maestro, agrup_maestro, muni_union, agrup_union, threshold=85, threshold_mun=90):
    
    count_maestro = df_maestro.shape[0]
    count_union = df_union.shape[0]
    print('num registros maestro: {}'.format(count_maestro))
    print('num registros fichero para unir: {}'.format(count_union))
    
    #fuzzy por prov/ccaa
    maestro_agrup = pd.DataFrame(df_maestro[agrup_maestro].drop_duplicates())
    print('num agrupaciones maestro: {}'.format(maestro_agrup.shape))
    union_agrup = pd.DataFrame(df_union[agrup_union].drop_duplicates())
    print('num agrupaciones union: {}'.format(union_agrup.shape))

    agrup_match = fuzzy_merge(maestro_agrup, union_agrup, agrup_maestro, agrup_union, threshold=threshold)
    print('num agrupaciones tras match: {}'.format(agrup_match.shape))
    num_matches = (agrup_match[agrup_match['matches']!='']['matches'].nunique())
    if (num_matches != agrup_match[agrup_maestro].nunique()):
        print('no se han cruzado todos los valores:')
        print(agrup_match[agrup_match['matches']==''])

        #se genera un nuevo df con el match y el fichero de union
    df_union_agrup_match = pd.merge(df_union, agrup_match, left_on=agrup_union, right_on='matches', how='left')
    if (count_union != df_union_agrup_match.shape[0]):
        print('el fichero de union no dispone de todos los valores para cruzar')    

    df_union_agrup_match.drop(columns=['matches'], inplace=True)
    
    #fuzzy por municipio
    df_maestro['muni_agrup'] = df_maestro[muni_maestro]+"/"+df_maestro[agrup_maestro]
    df_union_agrup_match['muni_agrup'] = df_union_agrup_match[muni_union]+"/"+df_union_agrup_match[agrup_maestro]
   
    muni_match = pd.DataFrame()
    maestro_count = 0
    for agrup in list(maestro_agrup[agrup_maestro]):
        print(agrup)
        maestro_agrup_sel = df_maestro[df_maestro[agrup_maestro] == agrup].copy()
        maestro_count = maestro_agrup_sel.shape[0] + maestro_count
        union_agrup_sel = df_union_agrup_match[df_union_agrup_match[agrup_maestro] == agrup].copy()
        if union_agrup_sel.empty == True:
            print(f'no existe para {agrup}')
        
        else:
            aux = fuzzy_merge(maestro_agrup_sel, union_agrup_sel, 'muni_agrup', 'muni_agrup', threshold=threshold_mun)
            num_matches = (aux[aux['matches']!='']['matches'].nunique())
            if (num_matches != aux[muni_maestro].nunique()):
                print('no se han cruzado todos los valores:')
                #print(aux[aux['matches'].isnull()]) 
            muni_match = pd.concat([muni_match, aux])
        
        #comprobacion registros    
    print('num muni maestro: {}'.format(maestro_count))
    if maestro_count != muni_match.shape[0]:
        print("se han perdido registros")       
        
    muni_match.drop(columns='muni_agrup', inplace=True)
    df_union_agrup_match.drop(columns=[agrup_maestro], inplace=True)

        #se genera un nuevo df con el match y el fichero de union
    df_union_muni_match = pd.merge(muni_match, df_union_agrup_match, left_on='matches', right_on='muni_agrup', how='left')
    if (count_union != df_union_muni_match.shape[0]):
        print('el fichero de union no dispone de todos los valores para cruzar por municipio')        

    #merge maestro con fichero union
    #df_tras_union = pd.merge(df_maestro, df_union_muni_match, left_on=muni_maestro, right_on='matches', how='left')
    
    print('columnas final: {}'.format(df_union_muni_match.columns))
        #borramos columnas que sobran
    if muni_maestro != muni_union:
        df_union_muni_match.drop(columns=muni_union, inplace=True)
    if agrup_maestro != agrup_union:
        df_union_muni_match.drop(columns=agrup_union, inplace=True)
    df_union_muni_match.drop(columns=['matches', 'muni_agrup'], inplace=True)
    print('num registros tras union: {}'.format(df_union_muni_match.shape))
    
    return df_union_muni_match

In [24]:
def union_cod_postal(maestro_nivel_cod_postal_1, df_union_2, df_unido_3, cod_postal_1, cod_postal_2, muni_prov, oper_agrup):
    union_cp = pd.merge(maestro_nivel_cod_postal_1, df_union_2, left_on=cod_postal_1, right_on=cod_postal_2, how='left')
    union_cp.drop(columns=[cod_postal_1, cod_postal_2], inplace=True)
    union_cp = union_cp.groupby(by=muni_prov, as_index=False).agg([oper_agrup]).reset_index()
    union_cp.columns = union_cp.columns.droplevel(1)  
    df = pd.merge(df_unido_3, union_cp, left_on=muni_prov, right_on=muni_prov, how='left')
    return df

#### Carga y union de ficheros

In [25]:
os.getcwd()

'/home/patcalsi/projects/Exodus_urbano/data/patri'

In [26]:
os.listdir()

['sociedad_seguridad_guardia_civil_2019_limpio.csv',
 'maestro_nivel_municipio_ori.csv',
 'union_ficheros.ipynb',
 'maestro_nivel_ine.csv',
 'sociedad_educacion_universidad_2018_limpio.csv',
 'maestro_nivel_cod_postal.csv',
 'union_ficheros2.ipynb',
 'sociedad_sanidad_hospitales_2019_limpio.csv',
 'union_patri.csv',
 'sociedad_educacion_colegios_2020_limpio.csv',
 'sociedad_sanidad_centros_salud_2019_limpio.csv',
 'maestro_nuevo.csv',
 'maestro_nivel_municipio.csv',
 'union_ficheros_todos.ipynb',
 'creacion_maestros.ipynb',
 '.ipynb_checkpoints',
 '.~lock.Empresas-SICTED2.xls#',
 'economia_turismo_establecimientos_20XX_limpio.csv',
 '.~lock.union_patri.csv#',
 'tratamiento',
 'sociedad_usos_suelo_2019_limpio.csv',
 'manual.txt',
 'aaaa.csv',
 'sociedad_cultura_bibliotecas_2017_limipio.csv',
 'listado-codigos-postales-con-LatyLon.csv',
 'sociedad_educacion_nivel_formacion_ccaa_2018.csv',
 'maestro_nivel_todo.csv']

## Fichero maestro

### Creacion maestro nivel cod postal

In [27]:
maestro_nivel_cod_postal = pd.read_csv("maestro_nivel_cod_postal.csv", index_col=0, dtype={'postal_code':object})

### Creacion maestro nivel municipio

In [29]:
maestro_nivel_municipio_ori = pd.read_csv("maestro_nivel_municipio_ori.csv", index_col=0)
maestro_nivel_municipio_ori.shape

(6067, 9)

## Ficheros especificos

### Ficheros Patri

##### Bibliotecas

In [25]:
bibliotecas = pd.read_csv("sociedad_cultura_bibliotecas_2017_limipio.csv", index_col=0)
bibliotecas.shape

(3323, 4)

In [26]:
bibliotecas.head(2)

,municipio_limpio,ccaa,provincia,total_bibliotecas
0,alaior,illes_balears,illes_balears,1
1,alaro,illes_balears,illes_balears,1


In [27]:
bibliotecas[bibliotecas['municipio_limpio']=='palma']
# idioma municipio = es

,municipio_limpio,ccaa,provincia,total_bibliotecas
38,palma,illes_balears,illes_balears,22


In [28]:
bibliotecas.rename(columns={'ccaa': 'ccaa_limpio', 'total_bibliotecas': 'num_bibliotecas'}, inplace=True)
bibliotecas.drop(columns=['provincia'], inplace=True)

In [29]:
# comprobacion duplicados
bibliotecas[bibliotecas.duplicated()]

,municipio_limpio,ccaa_limpio,num_bibliotecas


In [30]:
union1 = union_muni(maestro_nivel_municipio_ori, bibliotecas, 'cities', 'ccaa', 'municipio_limpio', 'ccaa_limpio', threshold=85)

num registros maestro: 6067
num registros fichero para unir: 3323
num agrupaciones maestro: (19, 1)
num agrupaciones union: (19, 1)
num agrupaciones tras match: (19, 2)
pais_vasco
no se han cruzado todos los valores:
castilla_la_mancha
no se han cruzado todos los valores:
comunitat_valenciana
no se han cruzado todos los valores:
region_de_murcia
no se han cruzado todos los valores:
andalucia
no se han cruzado todos los valores:
castilla_y_leon
no se han cruzado todos los valores:
extremadura
no se han cruzado todos los valores:
illes_balears
no se han cruzado todos los valores:
cataluna
no se han cruzado todos los valores:
cantabria
no se han cruzado todos los valores:
la_rioja
no se han cruzado todos los valores:
galicia
no se han cruzado todos los valores:
comunidad_de_madrid
no se han cruzado todos los valores:
aragon
no se han cruzado todos los valores:
comunidad_foral_de_navarra
no se han cruzado todos los valores:
principado_de_asturias
no se han cruzado todos los valores:
canari

In [37]:
union1[union1['cities']=='soneja']

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,num_bibliotecas
1059,soneja,soneja,castellon,castellon,comunidad_valenciana,comunitat_valenciana,soneja/castellon,Castellón/Castelló,Soneja,1.0


In [38]:
union1[~union1['num_bibliotecas'].isnull()].shape

(3792, 10)

In [39]:
union1.shape

(6067, 10)

##### Hospitales

In [40]:
hospitales = pd.read_csv("sociedad_sanidad_hospitales_2019_limpio.csv", index_col=0, dtype={'cp': object})
hospitales.shape

(613, 2)

In [41]:
hospitales.head(2)

,cp,uds_hospiatales
0,01002,1
1,01006,1


In [42]:
maestro_nivel_cod_postal.columns

Index(['postal_code', 'cities_es/province'], dtype='object')

In [43]:
hospitales.rename(columns={'uds_hospiatales': 'num_hospitales'}, inplace=True)

In [45]:
union2 = union_cod_postal(maestro_nivel_cod_postal, hospitales, union1, 'postal_code', 'cp', 'cities_es/province', 'sum')
union2.shape

(6067, 11)

In [46]:
union2.head()

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,num_bibliotecas,num_hospitales
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,1.0,0.0
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,1.0,0.0
2,aramaio,aramaio,alava,araba,pais_vasco,pais_vasco,aramaio/araba,Araba/Álava,Aramaio,1.0,0.0
3,artziniega,artziniega,alava,araba,pais_vasco,pais_vasco,artziniega/araba,Araba/Álava,Artziniega,1.0,0.0
4,arminon,arminon,alava,araba,pais_vasco,pais_vasco,arminon/araba,Araba/Álava,Armiñón,NaN,0.0


In [47]:
union2[union2['num_hospitales']>0].shape

(339, 11)

##### Centros salud

In [48]:
centros_salud = pd.read_csv("sociedad_sanidad_centros_salud_2019_limpio.csv", index_col=0, dtype={'cp': object})
centros_salud.shape

(8316, 2)

In [49]:
centros_salud.head(2)

,cp,num_centros_salud
0,01001,1
1,01002,2


In [51]:
union3 = union_cod_postal(maestro_nivel_cod_postal, centros_salud, union2, 'postal_code', 'cp', 'cities_es/province', 'sum')
union3.shape

(6067, 12)

In [52]:
union3[union3['num_centros_salud']>0].shape

(5881, 12)

##### Colegios

In [53]:
colegios = pd.read_csv("sociedad_educacion_colegios_2020_limpio.csv", index_col=0, dtype={'codigo_postal': object})

In [54]:
colegios.shape

(7838, 5)

In [59]:
colegios.head(2)

,municipio_limpio,provincia_limpio,num_colegios_privados,num_colegios_publicos
0,a_abelaira,lugo,0.0,2.0
1,a_abelleira,pontevedra,1.0,0.0


In [56]:
colegios[colegios['codigo_postal']=='01450']
#codigos postales erroneos, se une por municipio/provincia y cambiando de idioma

,codigo_postal,municipio_limpio,provincia,num_colegios_privados,num_colegios_publicos


In [57]:
colegios[colegios['municipio_limpio']=='alegria_dulantzi']
# idioma municipio = es

,codigo_postal,municipio_limpio,provincia,num_colegios_privados,num_colegios_publicos
81,01240,alegria_dulantzi,araba,NaN,3.0


In [58]:
colegios.drop(columns=['codigo_postal'], inplace=True)
colegios.rename(columns={'provincia': 'provincia_limpio'}, inplace=True)
colegios = colegios.groupby(by=['municipio_limpio','provincia_limpio'], as_index=False).sum()
colegios.shape

(6331, 4)

In [60]:
union4 = union_muni(union3, colegios, 'cities_es', 'province', 'municipio_limpio', 'provincia_limpio', threshold=85, threshold_mun=90)

num registros maestro: 6067
num registros fichero para unir: 6331
num agrupaciones maestro: (52, 1)
num agrupaciones union: (52, 1)
num agrupaciones tras match: (52, 2)
araba
no se han cruzado todos los valores:
gipuzkoa
no se han cruzado todos los valores:
bizkaia
no se han cruzado todos los valores:
albacete
no se han cruzado todos los valores:
ciudad_real
no se han cruzado todos los valores:
cuenca
no se han cruzado todos los valores:
guadalajara
no se han cruzado todos los valores:
toledo
no se han cruzado todos los valores:
alicante
no se han cruzado todos los valores:
castellon
no se han cruzado todos los valores:
valencia
no se han cruzado todos los valores:
murcia
no se han cruzado todos los valores:
almeria
no se han cruzado todos los valores:
cadiz
no se han cruzado todos los valores:
cordoba
no se han cruzado todos los valores:
granada
no se han cruzado todos los valores:
huelva
no se han cruzado todos los valores:
jaen
no se han cruzado todos los valores:
malaga
no se han c

In [63]:
union4[union4['num_colegios_publicos']>0.0].shape

(4850, 14)

In [61]:
union4.columns

Index(['cities_es', 'cities', 'province_es', 'province', 'ccaa_es', 'ccaa',
       'cities_es/province', 'provincia_original', 'municipio_original',
       'num_bibliotecas', 'num_hospitales', 'num_centros_salud',
       'num_colegios_privados', 'num_colegios_publicos'],
      dtype='object')

In [62]:
union4.shape

(6067, 14)

##### Guardia Civil

In [87]:
guardia_civil = pd.read_csv("sociedad_seguridad_guardia_civil_2019_limpio.csv", index_col=0, dtype={'cp': object})

In [88]:
guardia_civil.shape

(2237, 4)

In [89]:
guardia_civil.head(5)
#codigo postal no valido

,cp,municipio_limpio,provincia,uds_guardia_civil
0,10003,caceres,caceres,1
1,1010,vitoria,alava,1
2,10100,miajadas,caceres,1
3,10110,madrigalejo,caceres,1
4,10120,logrosan,caceres,1


In [90]:
guardia_civil[guardia_civil['municipio_limpio']=='benicasim']
# idioma municipio = es

,cp,municipio_limpio,provincia,uds_guardia_civil
153,12560,benicasim,castellon,1


In [91]:
guardia_civil.drop(columns=['cp'], inplace=True)
guardia_civil.rename(columns={'provincia': 'provincia_limpio', 'uds_guardia_civil': 'num_guardia_civil'}, inplace=True)
guardia_civil = guardia_civil.groupby(by=['municipio_limpio','provincia_limpio'], as_index=False).sum()
guardia_civil.shape

(2139, 3)

In [139]:
union5 = union_muni(union4, guardia_civil, 'cities_es', 'province_es', 'municipio_limpio', 'provincia_limpio', threshold=85, threshold_mun=90)

num registros maestro: 6067
num registros fichero para unir: 2139
num agrupaciones maestro: (52, 1)
num agrupaciones union: (52, 1)
num agrupaciones tras match: (52, 2)
alava
no se han cruzado todos los valores:
guipuzkoa
no se han cruzado todos los valores:
vizcaya
no se han cruzado todos los valores:
albacete
no se han cruzado todos los valores:
ciudad_real
no se han cruzado todos los valores:
cuenca
no se han cruzado todos los valores:
guadalajara
no se han cruzado todos los valores:
toledo
no se han cruzado todos los valores:
alicante
no se han cruzado todos los valores:
castellon
no se han cruzado todos los valores:
valencia
no se han cruzado todos los valores:
murcia
no se han cruzado todos los valores:
almeria
no se han cruzado todos los valores:
cadiz
no se han cruzado todos los valores:
cordoba
no se han cruzado todos los valores:
granada
no se han cruzado todos los valores:
huelva
no se han cruzado todos los valores:
jaen
no se han cruzado todos los valores:
malaga
no se han 

In [93]:
union5[~union5['num_guardia_civil'].isnull()].shape

(4356, 15)

In [94]:
union5.head(2)

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,num_bibliotecas,num_hospitales,num_centros_salud,num_colegios_privados,num_colegios_publicos,num_guardia_civil
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,1.0,0.0,1.0,0.0,3.0,NaN
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,1.0,0.0,1.0,3.0,6.0,NaN


In [95]:
union5.shape

(6067, 15)

##### Universidades

In [112]:
universidad = pd.read_csv('sociedad_educacion_universidad_2018_limpio.csv', index_col=0, dtype={'cp': object})

In [113]:
universidad.shape

(165, 6)

In [117]:
universidad.head(2)

,municipio_limpio,ccaa,num_universidades_privadas,num_universidades_publicas,num_tit_uni_privadas,num_tit_uni_publicas
0,albacete,castilla_la_mancha,NaN,1.0,NaN,52.0
1,alcala_de_henares,comunidad_de_madrid,1.0,1.0,5.0,114.0


In [115]:
#### universidad['ccaa'].drop_duplicates().sort_values()

In [116]:
universidad['ccaa'].replace("balears","illes_balears", inplace=True, regex=True)
universidad['ccaa'].replace("madrid","comunidad_de_madrid", inplace=True, regex=True)
universidad['ccaa'].replace("navarra","comunidad_foral_de_navarra", inplace=True, regex=True)
universidad['ccaa'].replace("asturias","principado_de_asturias", inplace=True, regex=True)
universidad['ccaa'].replace("rioja","la_rioja", inplace=True, regex=True)
universidad['ccaa'].replace("murcia","region_de_murcia", inplace=True, regex=True)

In [118]:
universidad.rename(columns={'ccaa': 'ccaa_limpio'}, inplace=True)
universidad.shape

(165, 6)

In [140]:
union6 = union_muni(union5, universidad, 'cities_es', 'ccaa', 'municipio_limpio', 'ccaa_limpio', threshold=85, threshold_mun=90)

num registros maestro: 6067
num registros fichero para unir: 165
num agrupaciones maestro: (19, 1)
num agrupaciones union: (17, 1)
num agrupaciones tras match: (19, 2)
no se han cruzado todos los valores:
         ccaa matches
6065    ceuta        
6066  melilla        
pais_vasco
no se han cruzado todos los valores:
castilla_la_mancha
no se han cruzado todos los valores:
comunitat_valenciana
no se han cruzado todos los valores:
region_de_murcia
no se han cruzado todos los valores:
andalucia
no se han cruzado todos los valores:
castilla_y_leon
no se han cruzado todos los valores:
extremadura
no se han cruzado todos los valores:
illes_balears
no se han cruzado todos los valores:
cataluna
no se han cruzado todos los valores:
cantabria
no se han cruzado todos los valores:
la_rioja
no se han cruzado todos los valores:
galicia
no se han cruzado todos los valores:
comunidad_de_madrid
no se han cruzado todos los valores:
aragon
no se han cruzado todos los valores:
comunidad_foral_de_navarra
n

In [1]:
union6.head(5)

NameError: name 'union6' is not defined

In [2]:
union6.shape

NameError: name 'union6' is not defined

##### Establecimientos

In [155]:
establecimientos = pd.read_csv("economia_turismo_establecimientos_20XX_limpio.csv", index_col=0, dtype={'codigo_postal': object})

In [156]:
establecimientos.shape

(2063, 35)

In [164]:
establecimientos.head(2)

,codigo_postal,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,campings,campos_de_golf_y_campos_de_pitch&putt,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
0,00000,0.0,2.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,00720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [158]:
# mismo codigo postal para mas de un municipio, se producen duplicados
establecimientos[establecimientos['codigo_postal']=='01118']

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
18,01118,bernedo,euskadi_pais_vasco,NaN,1.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,01118,lagran,euskadi_pais_vasco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,01118,urturi,euskadi_pais_vasco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
establecimientos[establecimientos['municipio_limpio']=='alcoi']
# municipio en 2 idiomas, se cruza por cp

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
163,03800,alcoi,comunitat_valenciana,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
165,03801,alcoi,comunitat_valenciana,1.0,NaN,NaN,NaN,NaN,2.0,NaN,...,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
167,03802,alcoi,comunitat_valenciana,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
170,03804,alcoi,comunitat_valenciana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [163]:
establecimientos.drop(columns=['municipio_limpio','ccaa'], inplace=True)
establecimientos = establecimientos.groupby(['codigo_postal'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else x.head(1))

In [165]:
union7 = union_cod_postal(maestro_nivel_cod_postal, establecimientos, union6, 'postal_code', 'codigo_postal', 'cities_es/province', 'sum')
union7.shape

(6065, 51)

In [166]:
union7[~union7['agencias_de_viajes'].isnull()].shape

(6065, 51)

##### Usos suelo

In [111]:
usos_suelo = pd.read_csv("sociedad_usos_suelo_2019_limpio.csv", index_col=0)

In [112]:
usos_suelo.shape

(7610, 12)

In [113]:
usos_suelo.head(2)

,municipio_limpio,provincia,superficie_urbana,uds_almacen,uds_comercial,uds_cultural,uds_hosteleria,uds_industrial,uds_deportivo,uds_oficinas,uds_religioso,uds_espectaculos
0,abengibre,albacete,"302,079",13,3,2,1,169,1,2,1,0
1,alatoz,albacete,"339,890",41,2,1,1,140,4,2,2,0


In [114]:
usos_suelo['provincia'].drop_duplicates()
# no esta el pais vasco - se deescarta

0                     albacete
87                    alicante
228                    almeria
331                      avila
579                    badajoz
744                    balears
811                  barcelona
1122                    burgos
1493                   caceres
1716                     cadiz
1761                 castellon
1896               ciudad_real
1998                   cordoba
2075                    coruna
2169                    cuenca
2407                    girona
2628                   granada
2802               guadalajara
3090                    huelva
3170                    huesca
3372                      jaen
3469                      leon
3680                    lleida
3911                     rioja
4085                      lugo
4152                    madrid
4331                    malaga
4434                    murcia
4479                   ourense
4571                  asturias
4649                  palencia
4840                    palmas
4874    

## Exportar fichero union

In [167]:
union7.shape

(6065, 51)

In [168]:
union7.head(3)

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,num_bibliotecas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,aramaio,aramaio,alava,araba,pais_vasco,pais_vasco,aramaio/araba,Araba/Álava,Aramaio,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [169]:
union7.to_csv("union_patri.csv")

### Ficheros Guille

##### Trafico

In [81]:
trafico = pd.read_csv('../Guille/recursos_trafico_carreteras_clean_2018.csv',index_col=0)
trafico.shape

(71, 4)

In [82]:
trafico.head(2)

,CCAA_name,Provincia_name,Total: Millones vehículos-km,porcentaje_carreteras_ccaa_y_estado
0,andalucia,almeria,3425.81,0.830417
1,andalucia,cadiz,4725.45,0.877934


In [97]:
trafico['Provincia_name'].replace("palmas_las","las_palmas", inplace=True, regex=True)
trafico.rename(columns={'Total: Millones vehículos-km': 'total_millones_vehiculos_km'}, inplace=True)
trafico.drop(columns=['CCAA_name'], inplace=True)
trafico.drop_duplicates(subset=['Provincia_name'], inplace=True)

In [99]:
union_agrup_guille_1 = union_agrup(maestro_nivel_municipio_ori, trafico, 'province_es', 'Provincia_name', threshold=85)

num registros maestro: 6067
num registros fichero para unir: 66
num agrupaciones maestro: (52, 1)
num agrupaciones union: (66, 1)
num agrupaciones tras match: (52, 2)
el fichero de union no dispone de todos los valores para cruzar
columnas final: Index(['Provincia_name', 'total_millones_vehiculos_km',
       'porcentaje_carreteras_ccaa_y_estado', 'province_es', 'matches'],
      dtype='object')
num registros tras union: (52, 3)


In [100]:
union_agrup_guille_1.head(2)

,total_millones_vehiculos_km,porcentaje_carreteras_ccaa_y_estado,province_es
0,3425.81,0.830417,almeria
1,4725.45,0.877934,cadiz


In [101]:
union_guille_1 = pd.merge(maestro_nivel_municipio_ori, union_agrup_guille_1, left_on='province_es',right_on='province_es', how='left')

In [102]:
# comprobacion duplicados
union_agrup_guille_1[union_agrup_guille_1.duplicated()]

,total_millones_vehiculos_km,porcentaje_carreteras_ccaa_y_estado,province_es


In [103]:
union_guille_1.head(2)

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,total_millones_vehiculos_km,porcentaje_carreteras_ccaa_y_estado
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,1869.3,0.832942
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,1869.3,0.832942
2,aramaio,aramaio,alava,araba,pais_vasco,pais_vasco,aramaio/araba,Araba/Álava,Aramaio,1869.3,0.832942
3,artziniega,artziniega,alava,araba,pais_vasco,pais_vasco,artziniega/araba,Araba/Álava,Artziniega,1869.3,0.832942
4,arminon,arminon,alava,araba,pais_vasco,pais_vasco,arminon/araba,Araba/Álava,Armiñón,1869.3,0.832942


In [104]:
union_guille_1.shape

(6067, 11)

##### Empresas_autobuses

In [106]:
empresas_autobuses = pd.read_csv('../Guille/recursos_empresas_autobuses_clean.csv',index_col=0)
empresas_autobuses.shape

(77, 2)

In [116]:
empresas_autobuses.head(2)

,bus_based_companies_count,municipio_limpio
0,5,barcelona
1,3,caceres


In [115]:
empresas_autobuses['municipio_limpio'] = empresas_autobuses['Municipio'].apply(doClean)
empresas_autobuses.drop(columns='Municipio', inplace=True)

In [128]:
empresas_autobuses.head()

,bus_based_companies_count,municipio_limpio
0,5,barcelona
1,3,caceres
2,3,jaen
3,3,soria
4,3,salamanca


In [134]:
union_guille_2 = union_solo_muni(union_guille_1, empresas_autobuses, 'cities_es', 'municipio_limpio', threshold=93)

num registros maestro: 6067
num registros fichero para unir: 77
num agrupaciones tras match: (6067, 12)
no se han cruzado todos los valores:
el fichero de union no dispone de todos los valores para cruzar
columnas final: Index(['bus_based_companies_count', 'municipio_limpio', 'cities_es', 'cities',
       'province_es', 'province', 'ccaa_es', 'ccaa', 'cities_es/province',
       'provincia_original', 'municipio_original',
       'total_millones_vehiculos_km', 'porcentaje_carreteras_ccaa_y_estado',
       'matches'],
      dtype='object')
num registros tras union: (6067, 12)


In [136]:
union_guille_2[~union_guille_2['bus_based_companies_count'].isnull()].shape

(66, 12)

##### Cobertura red

In [150]:
cobertura_media = pd.read_csv('../Guille/recursos_cobertura_media_clean_2018.csv',index_col=0)
cobertura_media.shape

(8124, 4)

In [151]:
cobertura_media.head(2)

,Provinicia,Municipio,Cod ESP,Calidad_Cobertura_fija&movil_%
0,araba,alegria_dulantzi,1001000100,65.00
1,araba,amurrio,1002000100,62.75


In [153]:
cobertura_media['Provinicia'].replace("balears_illes","illes_balears", inplace=True, regex=True)
cobertura_media['Provinicia'].replace("palmas_las","las_palmas", inplace=True, regex=True)
cobertura_media['Provinicia'].replace("rioja_la","la_rioja", inplace=True, regex=True)

In [154]:
cobertura_media.rename(columns={'Calidad_Cobertura_fija&movil_%': 'perc_calidad_cobertura_fija_movil'}, inplace=True)
cobertura_media.drop(columns=['Cod ESP'], inplace=True)
cobertura_media.drop_duplicates(inplace=True)

In [155]:
union_guille_3 = union_muni(union_guille_2, cobertura_media, 'cities_es', 'province', 'Municipio', 'Provinicia', threshold=80, threshold_mun=90)

num registros maestro: 6067
num registros fichero para unir: 8124
num agrupaciones maestro: (52, 1)
num agrupaciones union: (52, 1)
num agrupaciones tras match: (52, 2)
barcelona
no se han cruzado todos los valores:
caceres
no se han cruzado todos los valores:
jaen
no se han cruzado todos los valores:
soria
no se han cruzado todos los valores:
salamanca
no se han cruzado todos los valores:
granada
no se han cruzado todos los valores:
sevilla
no se han cruzado todos los valores:
ciudad_real
no se han cruzado todos los valores:
alicante
no se han cruzado todos los valores:
bizkaia
coruna
no se han cruzado todos los valores:
castellon
no se han cruzado todos los valores:
huelva
no se han cruzado todos los valores:
madrid
no se han cruzado todos los valores:
valencia
no se han cruzado todos los valores:
malaga
no se han cruzado todos los valores:
ourense
no se han cruzado todos los valores:
almeria
no se han cruzado todos los valores:
badajoz
no se han cruzado todos los valores:
albacete
n

In [158]:
union_guille_3[~union_guille_3['perc_calidad_cobertura_fija_movil'].isnull()].shape

(5999, 13)

In [161]:
union_guille_3.shape

(6067, 13)

##### Accesibilidad

In [171]:
accesibilidad_media_ccaa = pd.read_csv('../Guille/recursos_accesibilidad_ciudades_ccaa_2015.csv',index_col=0)
accesibilidad_media_ccaa.shape

(18, 5)

In [175]:
#trafico['Provincia_name'].replace("palmas_las","las_palmas", inplace=True, regex=True)
accesibilidad_media_ccaa.rename(columns={'NAME_1': 'ccaa_limpio'}, inplace=True)
accesibilidad_media_ccaa.drop(columns=['GID_0', 'NAME_0', 'GID_1'], inplace=True)
accesibilidad_media_ccaa.drop_duplicates(subset=['ccaa_limpio'], inplace=True)

In [189]:
accesibilidad_media_ccaa['ccaa_limpio'].replace("islas_canarias", "canarias", inplace=True, regex=True)

In [193]:
union_agrup_guille_4 = union_agrup(union_guille_3, accesibilidad_media_ccaa, 'ccaa_es', 'ccaa_limpio', threshold=85)

num registros maestro: 6067
num registros fichero para unir: 18
num agrupaciones maestro: (19, 1)
num agrupaciones union: (18, 1)
num agrupaciones tras match: (19, 2)
no se han cruzado todos los valores:
Empty DataFrame
Columns: [ccaa_es, matches]
Index: []
el fichero de union no dispone de todos los valores para cruzar
columnas final: Index(['ccaa_limpio', 'accessibility_mean', 'ccaa_es', 'matches'], dtype='object')
num registros tras union: (19, 2)


In [194]:
union_agrup_guille_4.head(2)

,accessibility_mean,ccaa_es
0,51.600938,andalucia
1,62.648308,aragon


In [195]:
union_guille_4 = pd.merge(maestro_nivel_municipio_ori, union_agrup_guille_4, left_on='ccaa_es',right_on='ccaa_es', how='left')

In [196]:
# comprobacion duplicados
union_guille_4[union_guille_4.duplicated()]

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,accessibility_mean


In [198]:
union_guille_4.head(2)

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,accessibility_mean
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,18.406372
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,18.406372


In [199]:
union_guille_4.shape

(6067, 10)

##### Tren

In [203]:
estaciones = pd.read_csv('../Guille/recursos_estaciones_renfe_clean_2019.csv',index_col=None)
estaciones.shape

(899, 5)

In [204]:
estaciones.head(2)

,poblacion,numero_estaciones,ave,cercanias,feve
0,madrid,39,SI,SI,NO
1,siero,17,NO,SI,SI
